Etape 1: Creation du DataSet

In [40]:
import pandas 
import numpy as np
import matplotlib.pyplot as plt


df_3features=pandas.read_csv("./datasets/Stress_temperature_step_humidity.csv", sep =',')
df_health=pandas.read_csv("./datasets/Sleep_health.csv", sep =',')
df_sleep=pandas.read_csv("./datasets/sayopillow/SaYoPillow.csv", sep =',')

df_health.drop(["Occupation","Blood Pressure"], axis = 1, inplace=True)





In [41]:
df_3features.tail()

,Humidity,Temperature,Step_count,Stress_Level
1996,21.82,90.82,96,1
1997,10.45,79.45,45,0
1998,27.22,96.22,135,2
1999,12.46,81.46,64,0
2000,16.87,85.87,50,1


In [42]:

df_3features.describe()

,Humidity,Temperature,Step_count,Stress_Level
count,2001.000000,2001.000000,2001.000000,2001.000000
mean,20.000000,89.000000,100.141429,1.104448
std,5.777833,5.777833,58.182948,0.771094
min,10.000000,79.000000,0.000000,0.000000
25%,15.000000,84.000000,50.000000,0.000000
50%,20.000000,89.000000,101.000000,1.000000
75%,25.000000,94.000000,150.000000,2.000000
max,30.000000,99.000000,200.000000,2.000000


In [43]:
df_3features.drop(["Step_count", "Temperature"], axis = 1, inplace=True)

In [32]:
df_health.head()



,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,6.1,6,42,6,Overweight,77,4200,NaN
1,2,Male,28,6.2,6,60,8,Normal,75,10000,NaN
2,3,Male,28,6.2,6,60,8,Normal,75,10000,NaN
3,4,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea
4,5,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea


In [33]:
valeurs_sleep_disorter = df_health['Sleep Disorder'].unique()

# Affiche les valeurs uniques
print(valeurs_sleep_disorter)

[nan 'Sleep Apnea' 'Insomnia']


In [35]:
valeurs_stress_lvl = df_health['Stress Level'].unique()

print(valeurs_stress_lvl)

[6 8 7 4 3 5]


In [44]:
df_health["Stress Level"] = df_health["Stress Level"].replace({ 3 : 0, 4 : 0, 5 : 1, 6 : 1, 7 : 2, 8 : 2})


In [48]:
df_health.head()

,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,6.1,6,42,1,Overweight,77,4200,NaN
1,2,Male,28,6.2,6,60,2,Normal,75,10000,NaN
2,3,Male,28,6.2,6,60,2,Normal,75,10000,NaN
3,4,Male,28,5.9,4,30,2,Obese,85,3000,Sleep Apnea
4,5,Male,28,5.9,4,30,2,Obese,85,3000,Sleep Apnea


In [49]:
df_3features.head()

,Humidity,Stress_Level
0,21.33,1
1,21.41,1
2,27.12,2
3,27.64,2
4,10.87,0


In [52]:
df_stress = pandas.merge(df_3features, df_health, left_on='Stress_Level', right_on='Stress Level', how='inner')

df_stress.head()

,Humidity,Stress_Level,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,21.33,1,1,Male,27,6.1,6,42,1,Overweight,77,4200,NaN
1,21.33,1,8,Male,29,7.8,7,75,1,Normal,70,8000,NaN
2,21.33,1,9,Male,29,7.8,7,75,1,Normal,70,8000,NaN
3,21.33,1,10,Male,29,7.8,7,75,1,Normal,70,8000,NaN
4,21.33,1,12,Male,29,7.8,7,75,1,Normal,70,8000,NaN
